In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
batch_size = 2
seq_len = 3
emb_size = 3
rnn_hidden_size = 5
print('BATCH SIZE:', batch_size)
print("SEQ LEN:", seq_len)
print("EMB SIZE:", emb_size)
print("RNN HIDDEN SIZE:", rnn_hidden_size)

BATCH SIZE: 2
SEQ LEN: 3
EMB SIZE: 3
RNN HIDDEN SIZE: 5


In [3]:
torch.manual_seed(0)

x_data = torch.randint(1, 9, (seq_len, batch_size))
print(x_data)

emb = nn.Embedding(9, emb_size)

# for param in emb.parameters():
#     print(param)

x_emb = emb(x_data)

print("x_emb")
print(x_emb)

rnn = nn.RNN(emb_size, rnn_hidden_size, bidirectional=True, bias = False)

# for name, param in rnn.named_parameters():
#     print(name, param.shape)
    
# print(rnn.weight_hh_l0)
# print()
# print(rnn.weight_hh_l0_reverse)

out, hidden = rnn(x_emb)
print("Pytorch results:")
print("out:")
print(out, out.shape)
print()
print("hidden:")
print(hidden, hidden.shape)

tensor([[5, 8],
        [6, 1],
        [4, 4]])
x_emb
tensor([[[ 0.1124,  0.6408,  0.4412],
         [-1.4689, -1.5867,  1.2032]],

        [[-0.2159,  0.7924, -0.2897],
         [ 0.4372,  0.4913, -0.2041]],

        [[-0.4927,  0.2484,  0.4397],
         [-0.4927,  0.2484,  0.4397]]], grad_fn=<EmbeddingBackward>)
Pytorch results:
out:
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897,  0.2490, -0.0043,
          -0.0176,  0.0240, -0.0313],
         [ 0.1044,  0.0042, -0.1612,  0.7877,  0.3083,  0.5819, -0.8315,
          -0.0468,  0.9113, -0.0991]],

        [[ 0.1415, -0.0160,  0.4295, -0.3774, -0.2531, -0.1587,  0.1502,
           0.3547, -0.3648, -0.2646],
         [ 0.2284,  0.3528, -0.2628, -0.2497, -0.1655, -0.3479,  0.2657,
           0.1215, -0.4595, -0.0252]],

        [[ 0.2052, -0.1044,  0.2178,  0.2974, -0.0797,  0.2642, -0.3012,
           0.2351,  0.2848, -0.2415],
         [ 0.2302, -0.0262,  0.2849, -0.1412, -0.0604,  0.2642, -0.3012,
           0.2351,  0.2848, 

In [11]:
tanh = nn.Tanh()

wii = rnn.weight_ih_l0
whh = rnn.weight_hh_l0

seq_len, batch_size , _ = x_emb.shape

outputs = []
h_t_f = torch.zeros(batch_size, rnn_hidden_size)

for i in range(seq_len):
    ii = x_emb[i] @ torch.transpose(wii, 0, 1)
    hh = h_t_f @ torch.transpose(whh, 0, 1)
    
    h_t_f = tanh(ii + hh)
    
    outputs.append(h_t_f)
    

print()
print("Output from forward pass:")
outputs = torch.stack(outputs)
print(outputs)


Output from forward pass:
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897],
         [ 0.1044,  0.0042, -0.1612,  0.7877,  0.3083]],

        [[ 0.1415, -0.0160,  0.4295, -0.3774, -0.2531],
         [ 0.2284,  0.3528, -0.2628, -0.2497, -0.1655]],

        [[ 0.2052, -0.1044,  0.2178,  0.2974, -0.0797],
         [ 0.2302, -0.0262,  0.2849, -0.1412, -0.0604]]],
       grad_fn=<StackBackward>)


In [32]:
x_emb_reverse = torch.flip(x_emb, [0, 1])

wii_reverse = rnn.weight_ih_l0_reverse
whh_reverse = rnn.weight_hh_l0_reverse

back_outputs = []
h_t_b = torch.zeros(batch_size, rnn_hidden_size)

for i in range(seq_len):
    ii = x_emb_reverse[i] @ torch.transpose(wii_reverse, 0, 1)
    hh = h_t_b @ torch.transpose(whh_reverse, 0, 1)
    
    h_t_b = tanh(ii + hh)
    
    back_outputs.append(h_t_b)

    
# print("raw back outputs:")
# print(back_outputs)
# print()
print("Output from backward pass:")
back_outputs = torch.flip(torch.stack(back_outputs), [0, 1])
print(back_outputs)

Output from backward pass:
tensor([[[ 0.2490, -0.0043, -0.0176,  0.0240, -0.0313],
         [ 0.5819, -0.8315, -0.0468,  0.9113, -0.0991]],

        [[-0.1587,  0.1502,  0.3547, -0.3648, -0.2646],
         [-0.3479,  0.2657,  0.1215, -0.4595, -0.0252]],

        [[ 0.2642, -0.3012,  0.2351,  0.2848, -0.2415],
         [ 0.2642, -0.3012,  0.2351,  0.2848, -0.2415]]],
       grad_fn=<FlipBackward>)


In [6]:
print()
print("Output from forward pass:")
# outputs = torch.stack(outputs)
print(outputs)

print()
print("Output from backward pass:")
# back_outputs = torch.flip(torch.stack(back_outputs), [0, 1])
print(back_outputs)


Output from forward pass:
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897],
         [ 0.1044,  0.0042, -0.1612,  0.7877,  0.3083]],

        [[ 0.1415, -0.0160,  0.4295, -0.3774, -0.2531],
         [ 0.2284,  0.3528, -0.2628, -0.2497, -0.1655]],

        [[ 0.2052, -0.1044,  0.2178,  0.2974, -0.0797],
         [ 0.2302, -0.0262,  0.2849, -0.1412, -0.0604]]],
       grad_fn=<StackBackward>)

Output from backward pass:
tensor([[[ 0.2490, -0.0043, -0.0176,  0.0240, -0.0313],
         [ 0.5819, -0.8315, -0.0468,  0.9113, -0.0991]],

        [[-0.1587,  0.1502,  0.3547, -0.3648, -0.2646],
         [-0.3479,  0.2657,  0.1215, -0.4595, -0.0252]],

        [[ 0.2642, -0.3012,  0.2351,  0.2848, -0.2415],
         [ 0.2642, -0.3012,  0.2351,  0.2848, -0.2415]]],
       grad_fn=<FlipBackward>)


In [9]:
my_final_output = torch.cat((outputs, back_outputs), -1)
print("pytorch final output:")
print(out)
print()
print("my final output: ")
print(my_final_output)

pytorch final output:
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897,  0.2490, -0.0043,
          -0.0176,  0.0240, -0.0313],
         [ 0.1044,  0.0042, -0.1612,  0.7877,  0.3083,  0.5819, -0.8315,
          -0.0468,  0.9113, -0.0991]],

        [[ 0.1415, -0.0160,  0.4295, -0.3774, -0.2531, -0.1587,  0.1502,
           0.3547, -0.3648, -0.2646],
         [ 0.2284,  0.3528, -0.2628, -0.2497, -0.1655, -0.3479,  0.2657,
           0.1215, -0.4595, -0.0252]],

        [[ 0.2052, -0.1044,  0.2178,  0.2974, -0.0797,  0.2642, -0.3012,
           0.2351,  0.2848, -0.2415],
         [ 0.2302, -0.0262,  0.2849, -0.1412, -0.0604,  0.2642, -0.3012,
           0.2351,  0.2848, -0.2415]]], grad_fn=<CatBackward>)

my final output: 
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897,  0.2490, -0.0043,
          -0.0176,  0.0240, -0.0313],
         [ 0.1044,  0.0042, -0.1612,  0.7877,  0.3083,  0.5819, -0.8315,
          -0.0468,  0.9113, -0.0991]],

        [[ 0.1415, -0.0160,  0.4295, -0.

In [36]:
print("pytorch final hidden:")
print(hidden)

print("My final hidden:")
print(h_t_f)
print(h_t_b)
# print(ht)

pytorch final hidden:
tensor([[[ 0.2052, -0.1044,  0.2178,  0.2974, -0.0797],
         [ 0.2302, -0.0262,  0.2849, -0.1412, -0.0604]],

        [[ 0.2490, -0.0043, -0.0176,  0.0240, -0.0313],
         [ 0.5819, -0.8315, -0.0468,  0.9113, -0.0991]]],
       grad_fn=<StackBackward>)
My final hidden:
tensor([[ 0.2052, -0.1044,  0.2178,  0.2974, -0.0797],
        [ 0.2302, -0.0262,  0.2849, -0.1412, -0.0604]], grad_fn=<TanhBackward>)
tensor([[ 0.5819, -0.8315, -0.0468,  0.9113, -0.0991],
        [ 0.2490, -0.0043, -0.0176,  0.0240, -0.0313]], grad_fn=<TanhBackward>)
